In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils

import jkutils.mpl_latex_template as mplt

import os 
os.environ["PATH"] += os.pathsep + '/nfs/homedirs/schuchaj/texlive/2020/bin/x86_64-linux'

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [2]:
collection = 'cert_collective'
collection_train = 'train_collective'

jk_config = {
    'username': 'schuchaj',
    'password': 'LBGqsHEY',
    'host': 'fs.kdd.in.tum.de',
    'port': 27017,
    'db_name': 'schuchaj'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)
col_train = db_utils.get_collection(collection_train, mongodb_config=jk_config)

In [3]:
def find_items(dataset, model, max_rad, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    
    db_filter = {
        'config.restrictions.dataset' : dataset,
        'config.restrictions.model' : model,
        'config.max_rad' : max_rad,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
    }
    if seed is not None:
        db_filter['config.seed'] = seed
    if col.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        cert_file_dir = exp['result']['cert_file']
        cert = torch.load(cert_file_dir)

        ret.append((exp, config, cert))
            
    return ret

In [4]:
def find_items_train(dataset, model, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    db_filter = {
        'config.dataset' : dataset,
        'config.model': model,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj
    }
    if seed is not None:
        db_filter['config.seed'] = seed

    if col_train.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col_train.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        
        run_id = config['overwrite']
        db_collection = config['db_collection']
        save_dir = config['save_dir']

        cert = torch.load(f'{save_dir}/{db_collection}_{run_id}', map_location=torch.device('cpu'))

        ret.append((exp, config, cert))
            
    return ret

In [5]:
def grid_from_exp(exp, collective=True):
    cert = exp[2]
    if collective:
        return cert['collective_grid']
    else:
        idx_test = cert['idx_test']
        return ((cert['grid_base'][idx_test] > 0.5).sum(axis=0))

In [6]:
# Styling
sns.set()
pal = sns.color_palette('colorblind', 2)

In [9]:
import torch
from robust_gcn.robust_gcn import RobustGCNModel, train, sparse_tensor
from robust_gcn.robust_gcn import certify as certify_attr
from robust_gcn.utils import train_val_test_split_tabular, load_npz
from matplotlib import pyplot as plt
from sparse_smoothing.utils import load_and_standardize
from sparse_smoothing.utils import split
from sparse_smoothing.cert_collective import collective_certificate
from time import localtime


# In[2]:


graph = load_and_standardize('citeseer')
A = graph.adj_matrix
X = graph.attr_matrix
z = graph.labels
K = z.max() + 1
N, D = X.shape

# In[3]:




/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/site-packages/gust-0.1-py3.7.egg/gust/preprocessing.py:238: UserWarning: 124 self loops removed
  warnings.warn("{0} self loops removed".format(num_self_loops))


In [11]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)


min_rad = np.min(
    [len(torch.load(f'/nfs/homedirs/schuchaj/gcn-robust-collective/base_{i}')['base_grid'].sum(axis=0))
     for i in range(5)
    ]
     )

grid_convex = np.zeros((5, min_rad))
for i in range(5):
    _, _, idx_test = split(
            labels=graph.labels, n_per_class=20, seed=i)

    x = torch.load(f'/nfs/homedirs/schuchaj/gcn-robust-collective/base_{i}')['base_grid'][idx_test].sum(axis=0, dtype='float')[:min_rad]
    x /= x[0]
    grid_convex[i] += x

grid_convex_collective = np.zeros((5, min_rad))
for i in range(5):
    x = torch.load(f'/nfs/homedirs/schuchaj/gcn-robust-collective/collective_{i}')['grid_collective'][0][0, :min_rad, 0].astype('float')
    x /= x[0]
    grid_convex_collective[i] += x

mean = np.mean(grid_convex_collective, axis=0)
std = np.std(grid_convex_collective, axis=0)
plt.plot(mean, label='Ours', color=pal[0])
plt.fill_between(np.arange(len(mean)), mean-std, mean+std, color=pal[0], alpha=0.15)


mean_naive = np.mean(grid_convex, axis=0)
std = np.std(grid_convex, axis=0)
plt.plot(mean_naive, label='Naive collective certificate', linestyle='--', color=pal[0], clip_on=False, zorder=3)
plt.fill_between(np.arange(len(mean_naive)), mean_naive-std, mean_naive+std, color=pal[0], alpha=0.15)

plt.legend(fontsize=7)

plt.xlabel('Attribute additions')
plt.ylabel('Certified ratio')
plt.ylim(0, 1)
plt.xlim(0, min_rad-1)

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/base_cert/attributes', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [18]:
base_grid = np.zeros((5, 10001))

for i in range(5):
    _, _, idx_test = split(
            labels=graph.labels, n_per_class=20, seed=i)

    x = torch.load(f'/nfs/homedirs/schuchaj/gcn-robust-collective/base_{i}')['base_grid'][idx_test].sum(axis=0, dtype='float')
    base_grid[i, :len(x)] += x

In [19]:
collective_grid = np.zeros((5, 10001))

for i in range(5):
    _, _, idx_test = split(
            labels=graph.labels, n_per_class=20, seed=i)

    x = torch.load(f'/nfs/homedirs/schuchaj/gcn-robust-collective/collective_{i}')['grid_collective'][0][0, :, 0]
    collective_grid[i, :len(x)] += x

In [22]:
base_grid = base_grid.astype('float')
base_grid /= base_grid.sum(axis=1, keepdims=True)
print((base_grid * np.arange(10001)).sum(axis=1).mean())

17.12394784248673


In [23]:
collective_grid[collective_grid == 1] = 0
collective_grid /= collective_grid.sum(axis=1, keepdims=True)
print((collective_grid * np.arange(10001)).sum(axis=1).mean())

971.3579089440096


In [25]:
times = [
    1 * 60 * 60 + 1 * 60 + 3,
    1 * 60 * 60 + 9 * 60 + 39,
    1 * 60 * 60 + 9 * 60 + 30,
    1 * 60 * 60 + 12 * 60 + 8,
    0 * 60 * 60 + 56 * 60 + 9,
]
print(np.mean(times) / 10001)

0.3941405859414059
